<a href="https://colab.research.google.com/github/syedanida/AI-Agents-using-Langraph/blob/main/LangGraph_AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langgraph openai langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
import os

# Set your API keys here
os.environ["OPENAI_API_KEY"] = "ABC"  # Replace with your OpenAI key
os.environ["LANGCHAIN_API_KEY"] = "XYZ"  # Replace with your LangSmith key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_Agent_Patterns"

### 🧠 1. Sequential Agent Workflow

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda, RunnableSequence

def step1(input):
    return f"Step 1 done with input: {input}"

def step2(input):
    return f"Step 2 processed: {input}"

sequential_chain = RunnableSequence(first=RunnableLambda(step1), last=RunnableLambda(step2))

result = sequential_chain.invoke("Start")
print("Sequential Result:", result)

Sequential Result: Step 2 processed: Step 1 done with input: Start


### 🤝 2. Multi-Agent Collaboration

In [ ]:
def researcher(input):
    return f"Research on '{input}': Found 3 key points."

def writer(input):
    return f"Writing blog using research: {input}"

multi_agent_chain = RunnableSequence(
    first=RunnableLambda(researcher),
    last=RunnableLambda(writer)
)

result = multi_agent_chain.invoke("AI Agents in LangGraph")
print("Multi-Agent Result:", result)

Multi-Agent Result: Writing blog using research: Research on 'AI Agents in LangGraph': Found 3 key points.


### 🔀 3. Conditional Branching

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableBranch

# Define the condition
def is_urgent(x):
    return "urgent" in x.lower()

# Define handlers
urgent_handler = RunnableLambda(lambda x: "Handled with HIGH priority 🚨")
normal_handler = RunnableLambda(lambda x: "Handled with normal workflow ✅")

# Correct usage: pass (condition, handler) as tuples, and default as a plain Runnable
conditional_chain = RunnableBranch(
    (is_urgent, urgent_handler),
    normal_handler  # Default branch: NOT in a tuple
)

# Test it out
print(conditional_chain.invoke("This is an urgent issue"))
print(conditional_chain.invoke("This is a regular update"))

Handled with HIGH priority 🚨
Handled with normal workflow ✅


### 🧠 4. Reactive Agent with Memory

In [ ]:
!pip install -q langchain langchain-community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Create memory and LLM
memory = ConversationBufferMemory()
llm = ChatOpenAI(temperature=0)

# Create conversation chain
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

# Example interaction
print(conversation.predict(input="Hi, I'm working on LangGraph."))
print(conversation.predict(input="What did I just say?"))
print(conversation.predict(input="Remind me what we discussed so far?"))

<ipython-input-16-fa0638f91358>:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-16-fa0638f91358>:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
<ipython-input-16-fa0638f91358>:10: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm working on LangGraph.
AI:

> Finished chain.
Hello! LangGraph is a fascinating project that focuses on language processing and graph theory. It involves analyzing and visualizing language data using graph structures to uncover patterns and relationships. How can I assist you with your work on LangGraph today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm working on L

### 🛠️ 5. Tool-Using Agent

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
import requests

def get_weather(city):
    return f"Dummy weather data for {city}"

tools = [
    Tool(
        name="WeatherTool",
        func=get_weather,
        description="Get current weather info for a city"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("What's the weather like in San Jose?")



> Entering new AgentExecutor chain...
I should use the WeatherTool to get the current weather in San Jose.
Action: WeatherTool
Action Input: San Jose
Observation: Dummy weather data for San Jose
Thought:I should try again to get the actual weather data for San Jose.
Action: WeatherTool
Action Input: San Jose
Observation: Dummy weather data for San Jose
Thought:I should try one more time to get the accurate weather data for San Jose.
Action: WeatherTool
Action Input: San Jose
Observation: Dummy weather data for San Jose
Thought:I should try a different approach to get the actual weather data for San Jose.
Action: WeatherTool
Action Input: San Jose
Observation: Dummy weather data for San Jose
Thought:I should check if there is an issue with the WeatherTool or if there is a problem with the data source for San Jose.
Action: Check WeatherTool status
Action Input: N/A
Observation: Check WeatherTool status is not a valid tool, try one of [WeatherTool].
Thought:I should try using a differen

'Unfortunately, I was unable to retrieve the accurate weather data for San Jose.'